In [1]:
import os
import glob
import pandas as pd
import plotly as py
import cufflinks as cf
import re
%matplotlib inline

In [2]:
base_path_to_csv = "/home/till/SharedFolder/trained_models/night-drive/weather_classifier/"
list_runs = ["without_cloudy", "with_cloudy"]
list_trials = ["train_A_over", "train_B_over", "train_C_over", "train_A_over_ganaug_025", "train_A_over_ganaug_050", "train_B_over_ganaug_025"]
list_selepochs = [[10, 10, 13, 10, 13, 10], # for each trial of run "without_cloudy"
                  [10, 10, 10, 10, 10, 10]] # for each trial of run "with_cloudy"
which_mode = "common"  # Options ["manual", "last", "best_train_dev", "best_f1"]
epoch_common = 16

In [3]:
df_sel_manual = pd.DataFrame()
df_all = pd.DataFrame()
for r, run in enumerate(list_runs):
    for t, trial in enumerate(list_trials):
        path_to_csv = os.path.join(base_path_to_csv,run,trial)
        csv_files = glob.glob(path_to_csv + "/*.csv")
        ### Combine CSVs into one DataFrame ###
        df = pd.DataFrame()
        for csv_file in csv_files:
            df = pd.concat([df, pd.read_csv(csv_file)], axis = 0)
            df = df.reset_index(drop = True)
        ### Some preprocessing ###
        #
        df = df.drop(columns = ["weights"])
        #
        df.data = df.data.apply(lambda x: x.split(os.sep)[-1].split(".json")[0].split("bdd100k_sorted_")[-1])
        df = df.sort_values(by = ["data", "epoch"])
        df = df.reset_index(drop = True)
        ### Load training log file ###
        train_log_file = glob.glob(path_to_csv + "/*.txt")
        assert(len(train_log_file) == 1)
        df_loss = pd.read_csv(train_log_file[0], sep = " ", header = None, skiprows = [0, 1])
        df_loss = df_loss.iloc[:, [1, 3, 7, 11]].rename(columns = {1: "epoch", 3: "batch", 7: "train_loss", 11: "train_dev_loss"})
        df_loss.epoch = df_loss.epoch.apply(lambda x: int(x.split("/")[0]))
        df_loss.batch = df_loss.batch.apply(lambda x: int(x.split("/")[0]))
        df_loss["global_batch"] = (df_loss.epoch - 1) * df_loss.batch.max() + df_loss.batch
        df_loss = df_loss.sort_values(by = ["epoch", "batch"]).set_index("global_batch", drop = True)
        ### Combine loss, scores
        df_combined = df_loss.groupby(by = ["epoch"]).median()
        df_combined = df_combined.drop(columns = ["batch"])
        
        # stash results for best manually selected epoch
        if which_mode == "manual":
            sel_epoch = list_selepochs[r][t]
        elif which_mode == "common":
            sel_epoch = epoch_common
            
        rw = df.query("epoch==@sel_epoch")
        rw.loc[:, "trial"] = list_trials[t]
        rw.loc[:, "run"] = list_runs[r]
        df_sel_manual = df_sel_manual.append(rw, ignore_index=True)
        # prmacro differences for each epoch
        # or all metrics using multi index
        df_all = pd.concat([df_all, df], axis=1)
        # stash results for best epoch according to: min train loss in train_dev
        
        # stash results for best epoch according to: f1_score


/home/till/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/till/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/till/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To ret

In [4]:
df_sel_manual.head(5)

,accuracy,data,epoch,f1_score_macro,f1_score_micro,f1_score_weighted,mcc,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro,run,trial
0,0.884021,test,16,0.789870,0.884021,0.886311,NaN,0.856009,0.937954,0.931407,0.964674,without_cloudy,train_A_over
1,0.925249,test_daytime,16,0.893564,0.925249,0.926708,NaN,0.952267,0.973406,0.985003,0.985885,without_cloudy,train_A_over
2,0.857895,test_night,16,0.672886,0.857895,0.860802,NaN,0.710860,0.903148,0.848927,0.945615,without_cloudy,train_A_over
3,0.917891,train_dev_A_over,16,0.917718,0.917891,0.917716,NaN,0.970932,0.972014,0.984406,0.984272,without_cloudy,train_A_over
4,0.882732,valid,16,0.780503,0.882732,0.886013,NaN,0.858515,0.944106,0.940590,0.968372,without_cloudy,train_A_over


In [5]:
### Create tables of summary stats of best epochs for every subplit
# rename train devs
df_sel_manual.data = df_sel_manual.data.apply(lambda x: re.sub(r"_[A-C]_[a-zA-Z0-9_]*", r"", x))

### Print results for best manual epochs gouped by split

In [6]:
# Selection (and order) of columns to be printed
cols2print = ["epoch", "trial", "run", 'accuracy','f1_score_macro', 'f1_score_micro', 'f1_score_weighted', 'pr_macro', 'pr_micro', 'roc_auc_macro', 'roc_auc_micro']

#### Valid-All

In [7]:
split = "valid"
df_cursplit = df_sel_manual.query("data==@split")
df_cursplit.reset_index(drop=True, inplace=True)
print("\n===", split, "=== Results for best manual epoch ===")
df_cursplit.loc[:, cols2print].head(12)


=== valid === Results for best manual epoch ===


,epoch,trial,run,accuracy,f1_score_macro,f1_score_micro,f1_score_weighted,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro
0,16,train_A_over,without_cloudy,0.882732,0.780503,0.882732,0.886013,0.858515,0.944106,0.940590,0.968372
1,16,train_B_over,without_cloudy,0.873711,0.784261,0.873711,0.879816,0.882327,0.940169,0.958277,0.966813
2,16,train_C_over,without_cloudy,0.871134,0.780609,0.871134,0.878037,0.879193,0.938038,0.956844,0.965469
3,16,train_A_over_ganaug_025,without_cloudy,0.873067,0.769566,0.873067,0.877726,0.860527,0.941974,0.945041,0.967025
4,16,train_A_over_ganaug_050,without_cloudy,0.859536,0.759398,0.859536,0.867916,0.854789,0.927930,0.942284,0.958492
5,16,train_B_over_ganaug_025,without_cloudy,0.913015,0.830930,0.913015,0.912893,0.883482,0.958544,0.954102,0.976597
6,16,train_A_over,with_cloudy,0.817500,0.738870,0.817500,0.820119,0.811910,0.872740,0.928556,0.949464
7,16,train_B_over,with_cloudy,0.832500,0.771680,0.832500,0.833933,0.847110,0.894839,0.944041,0.956511
8,16,train_C_over,with_cloudy,0.814500,0.746957,0.814500,0.818544,0.837722,0.880526,0.941987,0.950687
9,16,train_A_over_ganaug_025,with_cloudy,0.797500,0.728309,0.797500,0.802667,0.809344,0.851244,0.923156,0.940592


#### Valid-Night

In [8]:
split = "valid_night"
df_cursplit = df_sel_manual.query("data==@split")
df_cursplit.reset_index(drop=True, inplace=True)
print("\n===", split, "===")
df_cursplit.head(12)


=== valid_night ===


,accuracy,data,epoch,f1_score_macro,f1_score_micro,f1_score_weighted,mcc,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro,run,trial
0,0.860000,valid_night,16,0.674040,0.860000,0.865321,NaN,0.730106,0.914138,0.872428,0.952168,without_cloudy,train_A_over
1,0.851579,valid_night,16,0.702414,0.851579,0.862983,NaN,0.804142,0.909823,0.922231,0.951337,without_cloudy,train_B_over
2,0.843158,valid_night,16,0.698022,0.843158,0.857311,NaN,0.822347,0.908224,0.931460,0.950347,without_cloudy,train_C_over
3,0.847368,valid_night,16,0.666057,0.847368,0.856268,NaN,0.740674,0.909034,0.888470,0.950419,without_cloudy,train_A_over_ganaug_025
4,0.825263,valid_night,16,0.664759,0.825263,0.842611,NaN,0.744073,0.889227,0.887819,0.937381,without_cloudy,train_A_over_ganaug_050
5,0.909474,valid_night,16,0.764906,0.909474,0.905769,NaN,0.808342,0.951131,0.913236,0.972120,without_cloudy,train_B_over_ganaug_025
6,0.825000,valid_night,16,0.613250,0.825000,0.825910,0.494670,0.647975,0.853434,0.835077,0.940720,with_cloudy,train_A_over
7,0.854000,valid_night,16,0.692809,0.854000,0.856341,0.594282,0.745281,0.903168,0.895060,0.957802,with_cloudy,train_B_over
8,0.828000,valid_night,16,0.669217,0.828000,0.836603,0.563963,0.747094,0.877282,0.902636,0.948046,with_cloudy,train_C_over
9,0.785000,valid_night,16,0.578508,0.785000,0.801083,0.475326,0.640227,0.818890,0.844258,0.928841,with_cloudy,train_A_over_ganaug_025


In [9]:
print("\n===", split, "Difference to A ===")
df_cursplit_diff = df_cursplit.copy()
for col in df_cursplit.columns:
    if not isinstance(df_cursplit.loc[0,col], str):
        df_cursplit_diff.loc[0:5,col] = df_cursplit_diff.loc[0:5,col] - df_cursplit.loc[0,col]
        df_cursplit_diff.loc[6:11,col] = df_cursplit_diff.loc[6:11,col] - df_cursplit.loc[6,col]
df_cursplit_diff.loc[:,cols2print].head(12)


=== valid_night Difference to A ===


,epoch,trial,run,accuracy,f1_score_macro,f1_score_micro,f1_score_weighted,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro
0,0,train_A_over,without_cloudy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,train_B_over,without_cloudy,-0.008421,0.028373,-0.008421,-0.002339,0.074037,-0.004315,0.049803,-0.000831
2,0,train_C_over,without_cloudy,-0.016842,0.023981,-0.016842,-0.008010,0.092242,-0.005914,0.059032,-0.001822
3,0,train_A_over_ganaug_025,without_cloudy,-0.012632,-0.007983,-0.012632,-0.009054,0.010568,-0.005104,0.016042,-0.001750
4,0,train_A_over_ganaug_050,without_cloudy,-0.034737,-0.009281,-0.034737,-0.022711,0.013967,-0.024911,0.015392,-0.014788
5,0,train_B_over_ganaug_025,without_cloudy,0.049474,0.090866,0.049474,0.040447,0.078236,0.036993,0.040809,0.019951
6,0,train_A_over,with_cloudy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0,train_B_over,with_cloudy,0.029000,0.079559,0.029000,0.030431,0.097307,0.049733,0.059983,0.017081
8,0,train_C_over,with_cloudy,0.003000,0.055967,0.003000,0.010693,0.099119,0.023848,0.067559,0.007326
9,0,train_A_over_ganaug_025,with_cloudy,-0.040000,-0.034742,-0.040000,-0.024827,-0.007748,-0.034545,0.009181,-0.011879


#### Test-All

In [10]:
split = "test"
df_cursplit = df_sel_manual.query("data==@split")
df_cursplit.reset_index(drop=True, inplace=True)
print("\n===", split, "===")
df_cursplit.loc[:,cols2print].head(12)


=== test ===


,epoch,trial,run,accuracy,f1_score_macro,f1_score_micro,f1_score_weighted,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro
0,16,train_A_over,without_cloudy,0.884021,0.789870,0.884021,0.886311,0.856009,0.937954,0.931407,0.964674
1,16,train_B_over,without_cloudy,0.876933,0.788316,0.876933,0.882765,0.880058,0.936207,0.947441,0.964496
2,16,train_C_over,without_cloudy,0.865335,0.774133,0.865335,0.872352,0.876612,0.933659,0.948913,0.963231
3,16,train_A_over_ganaug_025,without_cloudy,0.868557,0.767692,0.868557,0.873275,0.849913,0.935402,0.932917,0.962594
4,16,train_A_over_ganaug_050,without_cloudy,0.852448,0.750737,0.852448,0.861052,0.846888,0.922186,0.930753,0.954944
5,16,train_B_over_ganaug_025,without_cloudy,0.899485,0.804651,0.899485,0.899883,0.872742,0.952064,0.942685,0.972704
6,16,train_A_over,with_cloudy,0.831500,0.759489,0.831500,0.834381,0.831204,0.888218,0.933961,0.955122
7,16,train_B_over,with_cloudy,0.854000,0.791609,0.854000,0.855620,0.847538,0.898213,0.943718,0.959400
8,16,train_C_over,with_cloudy,0.833500,0.768414,0.833500,0.837436,0.845954,0.885123,0.943420,0.954413
9,16,train_A_over_ganaug_025,with_cloudy,0.798000,0.731067,0.798000,0.803979,0.820457,0.860281,0.924937,0.942222


#### Test-Night

In [11]:
split = "test_night"
df_cursplit = df_sel_manual.query("data==@split")
df_cursplit.reset_index(drop=True, inplace=True)
print("\n===", split, "===")
df_cursplit.loc[:,cols2print].head(12)


=== test_night ===


,epoch,trial,run,accuracy,f1_score_macro,f1_score_micro,f1_score_weighted,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro
0,16,train_A_over,without_cloudy,0.857895,0.672886,0.857895,0.860802,0.710860,0.903148,0.848927,0.945615
1,16,train_B_over,without_cloudy,0.855789,0.704594,0.855789,0.866713,0.789148,0.899050,0.895270,0.946846
2,16,train_C_over,without_cloudy,0.837895,0.681493,0.837895,0.851636,0.789075,0.892318,0.898909,0.944507
3,16,train_A_over_ganaug_025,without_cloudy,0.833684,0.642071,0.833684,0.844048,0.710098,0.896407,0.858418,0.941146
4,16,train_A_over_ganaug_050,without_cloudy,0.818947,0.643788,0.818947,0.836891,0.721460,0.876704,0.862630,0.931007
5,16,train_B_over_ganaug_025,without_cloudy,0.896842,0.728117,0.896842,0.893917,0.770230,0.935322,0.884045,0.964180
6,16,train_A_over,with_cloudy,0.828000,0.610354,0.828000,0.826806,0.678786,0.872658,0.850359,0.947590
7,16,train_B_over,with_cloudy,0.867000,0.701427,0.867000,0.866862,0.742923,0.892606,0.890373,0.957618
8,16,train_C_over,with_cloudy,0.840000,0.655451,0.840000,0.842867,0.748267,0.869309,0.891179,0.948745
9,16,train_A_over_ganaug_025,with_cloudy,0.780000,0.582925,0.780000,0.796150,0.654635,0.818994,0.837807,0.924043


In [12]:
print("\n===", split, "Difference to A ===")
df_cursplit_diff = df_cursplit.copy()
for col in df_cursplit.columns:
    if not isinstance(df_cursplit.loc[0,col], str):
        df_cursplit_diff.loc[0:5,col] = df_cursplit_diff.loc[0:5,col] - df_cursplit.loc[0,col]
        df_cursplit_diff.loc[6:11,col] = df_cursplit_diff.loc[6:11,col] - df_cursplit.loc[6,col]
df_cursplit_diff.loc[:,cols2print].head(12)


=== test_night Difference to A ===


,epoch,trial,run,accuracy,f1_score_macro,f1_score_micro,f1_score_weighted,pr_macro,pr_micro,roc_auc_macro,roc_auc_micro
0,0,train_A_over,without_cloudy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,train_B_over,without_cloudy,-0.002105,0.031709,-0.002105,0.005911,0.078288,-0.004098,0.046343,0.001231
2,0,train_C_over,without_cloudy,-0.020000,0.008607,-0.020000,-0.009166,0.078215,-0.010830,0.049982,-0.001107
3,0,train_A_over_ganaug_025,without_cloudy,-0.024211,-0.030814,-0.024211,-0.016754,-0.000762,-0.006741,0.009491,-0.004469
4,0,train_A_over_ganaug_050,without_cloudy,-0.038947,-0.029098,-0.038947,-0.023911,0.010600,-0.026444,0.013703,-0.014608
5,0,train_B_over_ganaug_025,without_cloudy,0.038947,0.055231,0.038947,0.033115,0.059370,0.032174,0.035118,0.018565
6,0,train_A_over,with_cloudy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0,train_B_over,with_cloudy,0.039000,0.091073,0.039000,0.040056,0.064137,0.019948,0.040014,0.010029
8,0,train_C_over,with_cloudy,0.012000,0.045097,0.012000,0.016061,0.069482,-0.003349,0.040820,0.001155
9,0,train_A_over_ganaug_025,with_cloudy,-0.048000,-0.027429,-0.048000,-0.030656,-0.024151,-0.053665,-0.012551,-0.023547


In [13]:
# plot difference of pr_macro vs epoch


accuracy                    0.828
data                   test_night
epoch                          16
f1_score_macro           0.610354
f1_score_micro              0.828
f1_score_weighted        0.826806
mcc                      0.495474
pr_macro                 0.678786
pr_micro                 0.872658
roc_auc_macro            0.850359
roc_auc_micro             0.94759
run                   with_cloudy
trial                train_A_over
Name: 6, dtype: object